In [ ]:
# # TODO

# for id_ in G.nodes:
#     if '...","' in id_:
#         print(G.nodes[id_])

In [ ]:
import os
import sys

import networkx as nx

sys.path.append(os.path.abspath(".."))
import matplotlib.pyplot as plt

from yourtube.scraping import *

In [ ]:
# id_ = "HxGELmaqSHU"
# id_ = 'YOrJJ...","'
id_ = "Fa_un5wXC-ch"

In [ ]:
content = get_content(id_)
G = nx.DiGraph()

scrape_content(content, id_, G)